In [7]:
import os
import sys
import csv
import pandas as pd
import pandas_ta as ta

import ccxt  # noqa: E402

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   num_retries = 0
   try:
       num_retries += 1
       ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
       # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
       return ohlcv
   except Exception:
       if num_retries > max_retries:
           raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
   timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
   timedelta = limit * timeframe_duration_in_ms
   now = exchange.milliseconds()
   all_ohlcv = []
   fetch_since = since
   while fetch_since < now:
       ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
       fetch_since = (ohlcv[-1][0] + 1) if len(ohlcv) else (fetch_since + timedelta)
       all_ohlcv = all_ohlcv + ohlcv
       #if len(all_ohlcv):
           #print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
       #else:
           #print(len(all_ohlcv), 'candles in total from', exchange.iso8601(fetch_since))
   return exchange.filter_by_since_limit(all_ohlcv, since, None, key=0)


def scrape_candles_to_csv(exchange_id, max_retries, symbol, timeframe, since, limit):
   # instantiate the exchange by id
   exchange = getattr(ccxt, exchange_id)()
   # convert since from string to milliseconds integer if needed
   if isinstance(since, str):
       since = exchange.parse8601(since)
   # preload all markets from the exchange
   exchange.load_markets()
   # fetch all candles
   ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
   # save them to csv file
   #write_to_csv(filename, ohlcv)
   df = pd.DataFrame(ohlcv, dtype=float) 
   df.rename(columns={0: 'date', 1: 'open', 2: 'high',
                   3: 'low', 4: 'close', 5: 'volume'}, inplace=True)
   df['date'] = pd.to_datetime(df['date'], unit='ms')
   df['vwap'] = (df['open'] + 2 * df['high'] + 2*df['low'] + df['close'])/6
   save_name = symbol.split('/')[0]
   df['symbol'] = save_name
   df_aberration = df.ta.aberration()
   df_accbands = df.ta.accbands()
   df_rsi_series_14 = df.ta.rsi()
   df_er_series_10 = df.ta.er()
   df_ad_series = df.ta.ad()
   df_adosc_series_3_10 = df.ta.adosc()
   df_adx = df.ta.adx()
   df_alma_series_10_6 = df.ta.alma()
   df_macd = df.ta.macd()
   df_fisher = df.ta.fisher()
   df_cci_series_14 = df.ta.cci()
   df_zscore_series_30 =df.ta.zscore()
   df_kvo = df.ta.kvo()
   df_chop_series_14 = df.ta.chop()
   df_roc_series_10 = df.ta.roc()
   df_mom_series_10 = df.ta.mom()
   df_stoch = df.ta.stoch()
   df_cfo_series_9 = df.ta.cfo()
   df_ao_series_5 = df.ta.ao()
   df_aobv = df.ta.aobv()
   df_apo_series_12 = df.ta.apo()
   df_atr_14 = df.ta.atr()
   df_bbands = df.ta.bbands()
   df_bias_series_26 = df.ta.bias()
   df_bop_series = df.ta.bop()
   df_brar = df.ta.brar()
   df_cksp = df.ta.cksp()
   df_cmf_series_20 = df.ta.cmf()
   df_cmo_series_14 = df.ta.cmo()
   df_tema_series_10 = df.ta.tema()
   df_t3_series_10 = df.ta.t3()
   df_hma_series_10 = df.ta.hma()
   df_kama_series_10 = df.ta.kama()
   df_kc = df.ta.kc()
   df_tsi = df.ta.tsi()
   df_vhf_series_28 = df.ta.vhf()
   df_trima_series_10 = df.ta.trima()
   df_willr_series_14 = df.ta.willr()
   df_zlma_series_10 = df.ta.zlma()
   df_dema_series_10 = df.ta.dema()
   df_hwc = df.ta.hwc()
   df_new = df.copy()
   df_new = df_new.join(df_hwc)
   df_new = df_new.join(df_dema_series_10)
   df_new = df_new.join(df_zlma_series_10)
   df_new = df_new.join(df_willr_series_14)
   df_new = df_new.join(df_trima_series_10)
   df_new = df_new.join(df_vhf_series_28)
   df_new = df_new.join(df_tsi)
   df_new = df_new.join(df_kc)
   df_new = df_new.join(df_kama_series_10)
   df_new = df_new.join(df_t3_series_10)
   df_new = df_new.join(df_hma_series_10)
   df_new = df_new.join(df_tema_series_10)
   df_new = df_new.join(df_cmo_series_14)
   df_new = df_new.join(df_cmf_series_20)
   df_new = df_new.join(df_cksp)
   df_new = df_new.join(df_cci_series_14)
   df_new = df_new.join(df_brar)
   df_new = df_new.join(df_bop_series)
   df_new = df_new.join(df_bias_series_26)
   df_new = df_new.join(df_bbands)
   df_new = df_new.join(df_atr_14)
   df_new = df_new.join(df_apo_series_12)
   df_new = df_new.join(df_aobv)
   df_new = df_new.join(df_ao_series_5)
   df_new = df_new.join(df_cfo_series_9)
   df_new = df_new.join(df_stoch)
   df_new = df_new.join(df_mom_series_10)
   df_new = df_new.join(df_roc_series_10 )
   df_new = df_new.join(df_chop_series_14)
   df_new = df_new.join(df_kvo)
   df_new = df_new.join(df_zscore_series_30)
   df_new = df_new.join(df_fisher)
   df_new = df_new.join(df_macd)
   df_new = df_new.join(df_alma_series_10_6)
   df_new = df_new.join(df_adx)
   df_new = df_new.join(df_adosc_series_3_10)
   df_new = df_new.join(df_ad_series)
   df_new = df_new.join(df_er_series_10)
   df_new = df_new.join(df_rsi_series_14)
   df_new = df_new.join(df_accbands)
   df_new = df_new.join(df_aberration)

   path = 'crypto_data_1h_2017'
   csv_name = path + '/' + save_name + '.csv'
   df_new.to_csv(csv_name)
   #print('Saved', len(ohlcv), 'candles f

In [5]:
symbol_list = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'DOGE/USDT', 'SOL/USDT', 'XRP/USDT']

In [ ]:
for symbol in symbol_list:
    scrape_candles_to_csv('binance', 3, symbol, '1h', '2017-08-01T00:00:00Z', 100)

In [ ]:
import pandas_ta as ta
import pandas as pd
import ccxt

In [3]:
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '1h'
limit = 500
rsi_length = 100
since = '2017-08-01T00:00:00Z'
while True:
    try:
        if isinstance(since, str):
            since = exchange.parse8601(since)
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        print('--------------------------------------------------------------')
        if len(ohlcv):
            df = pd.DataFrame(ohlcv, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df = pd.concat([df, df.ta.rsi(length=rsi_length)], axis=1)
            print(df[-20:])
            print(exchange.iso8601 (exchange.milliseconds()))
    except Exception as e:
        print(type(e).__name__, str(e))

--------------------------------------------------------------
                   time     open     high      low    close      volume  \
480 2017-09-06 04:00:00  4448.10  4466.09  4365.73  4388.48   33.166056   
481 2017-09-06 05:00:00  4365.73  4437.29  4335.26  4416.17   47.486376   
482 2017-09-06 06:00:00  4416.54  4422.36  4335.26  4385.51   30.240164   
483 2017-09-06 07:00:00  4393.50  4505.96  4364.93  4461.29   55.415346   
484 2017-09-06 08:00:00  4476.13  4574.46  4474.20  4513.52   43.460837   
485 2017-09-06 09:00:00  4559.99  4574.46  4487.01  4492.66   37.027982   
486 2017-09-06 10:00:00  4497.81  4538.95  4469.82  4525.39   22.478306   
487 2017-09-06 11:00:00  4535.29  4597.79  4485.01  4548.13   72.327695   
488 2017-09-06 12:00:00  4544.59  4594.90  4490.04  4527.04  124.337025   
489 2017-09-06 13:00:00  4526.46  4547.77  4422.28  4502.77   48.475811   
490 2017-09-06 14:00:00  4490.01  4546.57  4466.20  4517.97   44.921193   
491 2017-09-06 15:00:00  4503.00  461

KeyboardInterrupt: 

In [4]:
# common constants

msec = 1000
minute = 60 * msec
hold = 30

# -----------------------------------------------------------------------------

exchange = ccxt.binance()

# -----------------------------------------------------------------------------

from_datetime = '2018-01-01 00:00:00'
from_timestamp = exchange.parse8601(from_datetime)

# -----------------------------------------------------------------------------

now = exchange.milliseconds()

# -----------------------------------------------------------------------------

data = []

while from_timestamp < now:

    try:

        print(exchange.milliseconds(), 'Fetching candles starting from', exchange.iso8601(from_timestamp))
        ohlcvs = exchange.fetch_ohlcv('ETH/BTC', '1m', from_timestamp)
        print(exchange.milliseconds(), 'Fetched', len(ohlcvs), 'candles')
        first = ohlcvs[0][0]
        last = ohlcvs[-1][0]
        print('First candle epoch', first, exchange.iso8601(first))
        print('Last candle epoch', last, exchange.iso8601(last))
        from_timestamp += len(ohlcvs) * minute
        data += ohlcvs

    except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout) as error:

        print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
        time.sleep(hold)

1723607051766 Fetching candles starting from 2018-01-01T00:00:00.000Z
1723607054264 Fetched 500 candles
First candle epoch 1514764800000 2018-01-01T00:00:00.000Z
Last candle epoch 1514794740000 2018-01-01T08:19:00.000Z
1723607054265 Fetching candles starting from 2018-01-01T08:20:00.000Z
1723607054350 Fetched 500 candles
First candle epoch 1514794800000 2018-01-01T08:20:00.000Z
Last candle epoch 1514824740000 2018-01-01T16:39:00.000Z
1723607054350 Fetching candles starting from 2018-01-01T16:40:00.000Z
1723607054423 Fetched 500 candles
First candle epoch 1514824800000 2018-01-01T16:40:00.000Z
Last candle epoch 1514854740000 2018-01-02T00:59:00.000Z
1723607054423 Fetching candles starting from 2018-01-02T01:00:00.000Z
1723607054562 Fetched 500 candles
First candle epoch 1514854800000 2018-01-02T01:00:00.000Z
Last candle epoch 1514884740000 2018-01-02T09:19:00.000Z
1723607054562 Fetching candles starting from 2018-01-02T09:20:00.000Z
1723607054669 Fetched 500 candles
First candle epoch 1

KeyboardInterrupt: 

In [5]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   num_retries = 0
   try:
       num_retries += 1
       ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
       # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
       return ohlcv
   except Exception:
       if num_retries > max_retries:
           raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')

In [13]:
#--------------------------------------------------------------------------------------------------------------------------
# Set the Parameters
timeframe = '1h'
symbol = 'BTC/USDT'
since = '2024-08-01T00:00:00Z'
limit = 20
max_retries = 3
exchange = getattr(ccxt, 'binance')()
timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
timedelta = limit * timeframe_duration_in_ms

#--------------------------------------------------------------------------------------------------------------------------
# Call the data from the exchange
if isinstance(since, str):
    since = exchange.parse8601(since)
now = exchange.milliseconds()
all_ohlcv = []
fetch_since = since

# Create a loop to make sure we get the latest data
while fetch_since < now:
    ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
    fetch_since = (ohlcv[-1][0] + 1) if len(ohlcv) else (fetch_since + timedelta)

    #Store the data
    all_ohlcv = all_ohlcv + ohlcv
    ohlcv = exchange.filter_by_since_limit(all_ohlcv, since, None, key=0)

    # Create the dataframe
    df = pd.DataFrame(ohlcv, dtype=float) 
    df.rename(columns={0: 'date', 1: 'open', 2: 'high', 3: 'low', 4: 'close', 5: 'volume'}, inplace=True)
    df['date'] = pd.to_datetime(df['date'], unit='ms')

    #Set the index for the vwap calculation
    df_timeindex = df.copy()
    df_timeindex = df_timeindex.set_index(df_timeindex['date'])
    df_timeindex.index = pd.to_datetime(df_timeindex.index)

    #Calculate the vwap 
    df_timeindex = df_timeindex.join(df_timeindex.ta.vwap(anchor='h'))
    df_timeindex.fillna(method='ffill', inplace=True)

    #calculate the RSI_14 based on the vwap
    df_timeindex = df_timeindex.join(ta.rsi(df_timeindex['VWAP_H']))

    last_row = df_timeindex.iloc[-1]
    previous_row = df_timeindex.iloc[-2]
    print(timeframe, symbol, '\tRSI_14 =', last_row['RSI_14'], last_row['date'], last_row['close'])


1h BTC/USDT 	RSI_14 = 33.70631322169243 2024-08-01 19:00:00 63422.01
1h BTC/USDT 	RSI_14 = 35.4670835742317 2024-08-02 15:00:00 63416.01
1h BTC/USDT 	RSI_14 = 40.246589803960056 2024-08-03 11:00:00 61919.38
1h BTC/USDT 	RSI_14 = 35.78006700301465 2024-08-04 07:00:00 60373.99
1h BTC/USDT 	RSI_14 = 15.892770832589854 2024-08-05 03:00:00 53890.87
1h BTC/USDT 	RSI_14 = 48.35591557512431 2024-08-05 23:00:00 54018.81
1h BTC/USDT 	RSI_14 = 62.84437899288016 2024-08-06 19:00:00 56765.48
1h BTC/USDT 	RSI_14 = 43.06775926337084 2024-08-07 15:00:00 56094.5
1h BTC/USDT 	RSI_14 = 64.18166531860325 2024-08-08 11:00:00 57320.51
1h BTC/USDT 	RSI_14 = 67.2939040000411 2024-08-09 07:00:00 60863.57
1h BTC/USDT 	RSI_14 = 51.573129702556244 2024-08-10 03:00:00 60437.84
1h BTC/USDT 	RSI_14 = 55.99417339834602 2024-08-10 23:00:00 60923.51
1h BTC/USDT 	RSI_14 = 27.39355457809773 2024-08-11 19:00:00 59621.11
1h BTC/USDT 	RSI_14 = 58.07500567276973 2024-08-12 15:00:00 60207.37
1h BTC/USDT 	RSI_14 = 41.926524737

In [ ]:
def scrape_candles_to_csv(exchange_id, max_retries, symbol, timeframe, since, limit):
   # instantiate the exchange by id
   exchange = getattr(ccxt, 'binance')()
   # convert since from string to milliseconds integer if needed
   if isinstance(since, str):
       since = exchange.parse8601(since)
   # preload all markets from the exchange
   exchange.load_markets()
   # fetch all candles
   ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
   # save them to csv file
   #write_to_csv(filename, ohlcv)
   df = pd.DataFrame(ohlcv, dtype=float) 
   df.rename(columns={0: 'date', 1: 'open', 2: 'high',
                   3: 'low', 4: 'close', 5: 'volume'}, inplace=True)

In [14]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   num_retries = 0
   try:
       num_retries += 1
       ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
       # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
       return ohlcv
   except Exception:
       if num_retries > max_retries:
           raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
   timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
   timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
   timedelta = limit * timeframe_duration_in_ms
   now = exchange.milliseconds()
   all_ohlcv = []
   fetch_since = since
   while fetch_since < now:
       ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
       fetch_since = (ohlcv[-1][0] + 1) if len(ohlcv) else (fetch_since + timedelta)
       all_ohlcv = all_ohlcv + ohlcv
       #if len(all_ohlcv):
           #print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
       #else:
           #print(len(all_ohlcv), 'candles in total from', exchange.iso8601(fetch_since))
   return exchange.filter_by_since_limit(all_ohlcv, since, None, key=0)


def scrape_candles_to_csv(exchange_id, max_retries, symbol, timeframe, since, limit):
   # instantiate the exchange by id
   exchange = getattr(ccxt, exchange_id)()
   # convert since from string to milliseconds integer if needed
   if isinstance(since, str):
       since = exchange.parse8601(since)
   # preload all markets from the exchange
   exchange.load_markets()
   # fetch all candles
   ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
   # save them to csv file
   #write_to_csv(filename, ohlcv)
   df = pd.DataFrame(ohlcv, dtype=float) 
   df.rename(columns={0: 'date', 1: 'open', 2: 'high',
                   3: 'low', 4: 'close', 5: 'volume'}, inplace=True)

In [21]:
df_new = scrape_candles_to_csv('binance', 3, 'BTC/USDT', '1h', '2023-08-01T00:00:00Z', 100)

In [23]:
df_new.head()

AttributeError: 'NoneType' object has no attribute 'head'